In [ ]:
save_name = 'DenseNet_Oversample_L2_Sum_PosWeight_5_Targets'

In [ ]:
import sys
import os, os.path

sys.path.append(os.path.join(os.getcwd() ,'/modules'))
root_path = "C:/git/Springboard-Public/Capstone Project 2/"
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    root_path = "/content/drive/My Drive/Capstone Project 2/"

print('Current Working Dir: ', os.getcwd())
print('Root Path: ', root_path)

# We need to set the working directory since we are using relative paths from various locations
if os.getcwd() != root_path:
  os.chdir(root_path)

In [ ]:
from modules.lib.StandardTraining import *
from modules.models.DenseNet import DenseNet

from torchsummary import summary

# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

%matplotlib inline

In [ ]:
force_cpu = True
device = torch.device('cuda' if ~force_cpu and torch.cuda.is_available() else 'cpu')
# Assume that we are on a CUDA machine, then this should print a CUDA device:
print(f'Working on device={device}')

In [ ]:
training_columns = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural_Effusion']
out_channels=len(training_columns)
net_kwargs = {'nr_classes':out_channels}
net = DenseNet(**net_kwargs)

net = nn.DataParallel(net)
net.to(device)


summary(net, (1, 224, 224))   

In [ ]:
standardTraining = StandardTraining(number_images=0, 
                                     batch_size=16, 
                                     learning_rate=1e-5, 
                                     num_epochs=6,
                                     device=device, 
                                     net=net,
                                     image_width = 224,
                                     image_height = 224,
                                     use_positivity_weights=True,
                                     observation_min_count=150,
                                     loss_reduction='sum',
                                     l2_reg=1e-1,
                                     target_columns=training_columns,
                                     augPercent=.4,
                                     save_path= f'notebooks/saved/{save_name}/',
                                     net_name = 'DenseNet',
                                     net_kwargs = net_kwargs)

In [ ]:
standardTraining.train()

In [ ]:
standardTraining.displayMetrics()

In [ ]:
standardTraining.displayEpochProgression()

In [ ]:
standardTraining.save()